# Реализация Fast R-CNN
В этом практическом уроке мы реализуем метод Fast R-CNN. Он будет во многом  похож на R-CNN. Однако, если в случае R-CNN пропозалы и нейросетевой классификатор были двумя независимыми вещами, то в случае Fast R-CNN пропозалы будут использоваться внутри сети, которая делает предсказание.

### Загрузка необходимых библиотек
Здесь мы загружаем различне библиотеки, включая TensoFlow.

В TensorFlow инициируем режим жадного (eager) выполнения и проверяем версию (должна быть 1.14)

In [ ]:
import random

import tensorflow as tf
tf.enable_eager_execution()
print('TensorFlow version:', tf.__version__)

## Архитектура Fast R-CNN
Перейдем сразу к архитектуре Fast R-CNN. Все остальыне части пайплайна не отличаются от R-CNN. Внутри Fast R-CNN по-прежнему будет находиться свёрточная нейросеть, которую необходимо обучить на задачу классификации, так же как в R-CNN. 

Однако инференс (детектирование) в Fast R-CNN работает иначе. Вместо того, чтобы запускать классификатор отдельно для каждого пропозала на картинке, мы сначала запускаем первую часть сети, которая извлекает признаки для всей картинки (один раз). А затем отображаем пропозалы в это признаковое пространство, вырезаем соответствующие кусочки, делаем ROI Pooling и прогоняем результат через вторую (классификационную) часть нейросети.

Получается, что у нас есть два режима работа модели: в режиме обучения наша модель работает (обучается) как простой классификатор, а в режиме детектирования модель работает описанным выше хитрым способом. В обоих режимах нам нужно использовать прямое распространение (в обучении это часть всего процесса), которое реализуется функцией `call()`. Таким образом нам придётся как-то организовать два типа поведения в рамках одной модели. Для этого давайте просто в функции `call()`, где и происходит прямое распространение, сделаем условное ветвление: если на вход пришли пропозалы, значит работаем в режиме инференса (предсказание), а если пропозалов нет, значит работаем в режиме обучения (как обычный классификатор). По сути, у нас получится динамический граф -- вызов тех или иных слоёв зависит от входных данных.

В режиме предсказания сначала применим свёрточные и пулинг слои. Затем получим координаты пропозалов в признаковом пространстве. Так как размерность признакового пространства уменьшилась по сравнению с входной картинкой из-за пулинга, то и пропозалы должны пропорционально уменьшиться. А так как у нас координаты пропозалов обычно задаются в относительнеых величинах, значит мы просто будем использовать размеры карт признаков (`feat.shape`) для перехода к абсолютным координатам. После этого вырезаем соответствующи кусочки из карт признаков (берём все каналы) и после приведения к нужному размеру (`roi_pool_size`) подаём этот тензор в полносвязные слои.

Однако, сделаем некоторое упрощение: вместо ROI Pooling, будем использовать похожую по эффекту операцию: простую билинейную интерполяцию -- то есть просто приводить тензоры к нужному фиксированному размеру с помощью resize.

Результаты всех предсказаний (если предсказанный клас не "фон") запишем в выходной список -- аналогично тому, как это было в R-CNN методе.

**[ЗАДАНИЕ 1]** Вопрос: Почему `roi_pool_size = (7, 7)`? При условии, что пространственные размеры картинок в классификационном датасете, на котором мы будем обучать эту модель - (28, 28).

In [ ]:
NUM_CLASSES = 11

class Model(tf.keras.Model):
    
    def __init__(self):
        super(Model, self).__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(32, (5, 5), activation='relu', padding='same')
        self.conv2 = tf.keras.layers.Conv2D(64, (5, 5), activation='relu', padding='same')
        self.fc1 = tf.keras.layers.Dense(256, activation='relu')
        self.fc2 = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
        self.max_pool = tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same')
        
    def call(self, inp, proposals=None):
        
        if proposals is None: # Режим обучения
          
            out = self.conv1(inp)
            out = self.max_pool(out)
            out = self.conv2(out)
            out = self.max_pool(out)
            out = tf.layers.flatten(out)
            out = self.fc1(out)
            out = self.fc2(out)
            return out
          
        else: # Режим предсказания
          
            assert inp.shape[0] == 1 # Только batch size = 1
            predictions = []
            roi_pool_size = (7, 7)
            
            # Извлечение признаков из всей картинки
            out = self.conv1(inp)
            out = self.max_pool(out)
            out = self.conv2(out)
            feat = self.max_pool(out)
            
            # Для каждого пропозала
            for proposal in proposals:
                
                # Отображение координат пространства изображения 
                # в координаты пространства признаков
                ry, rx, rh, rw = proposal        
                box_y = int(round(ry * int(feat.shape[1])))
                box_x = int(round(rx * int(feat.shape[2])))
                box_w = int(round(rw * int(feat.shape[2])))
                box_h = int(round(rh * int(feat.shape[1])))
                
                # Вырезаем признаки, относящиеся к пропозалу
                feat_sub = feat[:, box_y:box_y+box_h, box_x:box_x+box_w, :]
                
                # Аналог ROI Pooling
                feat_pooled = tf.image.resize(
                    feat_sub, 
                    (roi_pool_size[0], roi_pool_size[1]), 
                    tf.image.ResizeMethod.BILINEAR)
                
                # Финальная классификация
                out = tf.layers.flatten(feat_pooled)
                out = self.fc1(out)
                out = self.fc2(out)
                
                # Фильтрация класса "фон"
                assert out.shape[0] == 1 # Только batch size = 1
                pred = out[0]
                pred_cls = np.argmax(pred)
                if pred_cls != 10:
                    predictions.append([pred_cls] + proposal)               
        
            return predictions
    
model = Model()

### Обучение Fast R-CNN
Все остальные части пйплнайна обучения такие же, как и для R-CNN

**[ЗАДАНИЕ 2]**: Подготовьте необходимые данные: `train_x_cls`, `train_y_cls`, `test_x_det`, `test_y_det`, `test_proposals` точно так же, как это было в уроке про R-CNN. Обучите модель `model` на классификационном датасете, по аналогии с тем, как мы это делали для R-CNN. В режиме обучения в функцию `call()` не будут передаваться пропозалы, поэтому модель будет вести себя как обычный классификатор.

### Запуск детектирования объектов с помощью Fast R-CNN
Для запуска этого и последующих блоков понядобятся функции, реализованные в уроке, посвящённом R-CNN.

In [ ]:
idx = random.randint(0, 1000)
img = test_x_det[idx]
labels_true = test_y_det[idx]
proposals_img = test_proposals[idx]

preds = model(img[None, ...], proposals_img)

### Визуализация Ground-Truth

In [ ]:
show_prediction(img, labels_true)

### Визуализация пропозалов

In [ ]:
show_proposals(img, proposals_img)

### Визуализация детекций Fast R-CNN

In [ ]:
show_prediction(img, preds)

### Дополнительное задание
**[ЗАДАНИЕ 3]** Позапускайте Fast R-CNN для разных изображений из тестовой выборки, оцените качество. Попробуйте улучшить Fast R-CNN за счёт подбора гиперпараметров модели и обучения.